# multiply-ui

A GUI mock for the [Multiply](https://github.com/multiply-org) project based on
[Jupyter Widgets](https://ipywidgets.readthedocs.io) and [Bokeh](https://bokeh.pydata.org).

**Note**, you'll have to start the server first:
        
```
$ source activate multiply-ui
$ mui-server
```

Note for developers: the variable `debug_view` contains a snapshot of the Python function traceback recorded during the last encountered exception.


Import the Multiply user interface components, called *mui* from now on

In [1]:
from multiply_ui.ui import mui

Inspect all variables that Multiply can generate:

In [2]:
mui.vars

Id,Name,Units,Description
lai,Effective Leaf Area Index,m2/m2,Effective Leaf Area Index
cab,Leaf Chlorophyll Content,ug/cm2,Leaf Chlorophyll Content
cb,Leaf Senescent Material,,Leaf Senescent Material
car,Leaf Carotonoid Content,ug/cm2,Leaf Carotonoid Content
cw,Leaf Water Content,cm,Leaf Water Content
cdm,Leaf Dry Mass,g/cm2,Leaf Dry Mass
n,Leaf Structure Parameter,,Leaf Structure Parameter
ala,Average Leaf Angle,degrees,Average Leaf Angle
bsoil,Soil Brightness,,Soil Brightness Parameter
psoil,Soil Wetness,,Soil Wetness Parameter


Inspect all forward models that Multiply provides:

In [3]:
mui.models

Id,Name,Description,Author(s),
s2_prosail,PROSAIL for Sentinel-2,"Coupling of PROSPECT leaf optical properties model and SAIL canopy bidirectional reflectance model. It links the spectral variation of canopy reflectance, which is mainly related to leaf biochemical contents, with its directional variation, which is primarily related to canopy architecture and soil/vegetation contrast.",,More...


Inspect all input types that Multiply knows about:

In [4]:
mui.itypes

Id,Name,Time Range
S2_L1C,Sentinel-2 MSI L1C,"('06-23-2015', None)"


The entry point function `mui.sel_params()` brings up the main GUI form that can generate processing requests or submit processing requests directly. (We currently use a mock interface rather than the server. If you don't want that, set `mock=False` or omit the argument.)

In the following UI form, press *New Request*

In [5]:
mui.sel_params(identifier='req', name='my_request', mock=False)

Box(children=(Box(children=(HTML(value='<h2>Output Variables</h2>'),), layout=Layout(display='flex', flex_flow…

In [ ]:
req

We can now submit the request using the processing request's `submit` method. It takes a single argument that determines the name of the variable that will receive the job proxy object once the job execution has been accepted.

In [ ]:
req.submit('my_job', mock=True)

Let's look at the resulting job:

In [ ]:
my_job

Look at only the tasks ...

In [ ]:
my_job.tasks

... or only a specific task.

In [ ]:
my_job.tasks.get('Collecting static Data')

Now, let's monitor the progress of the job, using a dedicated UI form:

In [ ]:
mui.obs_job(my_job, mock=True)

We may also submit a processing request directly from the UI. Bring up the same form as in cell 6, but press *Submit Request* this time.

In [ ]:
mui.sel_params(identifier='my_other_job', name='my_other job\'s name', mock=True)

In [ ]:
my_other_job

Now that we have more than one job, we want to see te progresses of both of them. To this end, bring up the Jobs Observation UI form. This form shows all jobs that are registered in the MULTIPLY platform, along with their states. In this form, you have the option to cancel a job.

In [ ]:
mui.obs_jobs(mock=True)

Jobs can also be canceled via their `cancel` method.

In [ ]:
my_job.cancel(mock=True)

In [ ]:
my_job.req, my_job.result, my_job.status

The form can also be opened with values from a formerly stored processing request:

In [ ]:
mui.sel_params(my_job.req)

Show job summary

In [ ]:
mui.jobs()

JobID         |Status
--------------|-------
7816-23af-e273|5 %
6445-5drg-5hr5|74 %


In [ ]:
job = mui.job('7816-23af-e273')
job.cancel()

In [ ]:
job = mui.job('6445-5drg-5hr5')
job.result

*Render all results here...*

In [ ]:
# Open xarray dataset from result
ds = job.result.open()

Show map of LAI

In [ ]:
ds.lai.sel(time='2010-01-01', method='nearest').plot.imshow()

Show time-series of LAI

In [ ]:
ds.lai.sel(lon=10.2,lat=51.2, method='nearest').plot()

In [ ]:
debug_view